In [3]:
import pandas as pd
import re


In [4]:
df = pd.read_csv('data.csv')
variables = pd.read_csv('../../../variables.csv')

## Merge data with the variables file

In [5]:
df['Model_simplified'] = df['Model - new table'].apply(lambda x: re.findall('\w+', x))
df['Model_simplified'] = df['Model_simplified'].apply(lambda x: x[0])
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\gabriel.genelot\AppData\Local\Temp\ipykernel_30784\2487617104.py:1: SyntaxWarning: invalid escape sequence '\w'
  df['Model_simplified'] = df['Model - new table'].apply(lambda x: re.findall('\w+', x))


,Name,Equation number,Model - new table,Input variable,Output variables,Equation shape,Formule,Zotero reference,IPCC risks,Input categories,Output categories,Latex fragment,Vensim,Vensim name,Model_simplified
0,Total agriculture impact,A.1,FUND v.3.8,"Damage in agricultural production,Damage in ag...",Total agriculture impact,NaN,a1.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food production reduction,Food security reduction",Production,Production,"A_{t,r}=A_{t,r}^{r}+A_{t,r}^{l}+A_{t,r}^{f}",Oui,FUND total agricultural impact,FUND
1,Agriculture impact of the rate of climate change,A.2,FUND v.3.8,"Time,Region,Change in regional mean temperatur...",Damage in agricultural production,NaN,a2.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food security reduction,Food production reduction",", Temperature",Production,"A_{t,r}^{r}=\alpha_{r}\left(\frac{\Delta T_{t}...",Oui,NaN,FUND
2,Agriculture impact of the level of climate change,A.3,FUND v.3.8,"Time,Region,Change in regional mean temperatur...",Damage in agricultural production level,NaN,a3.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food production reduction,Food security reduction",", Temperature",Production,"A_{t,r}^{l}\,=\,\delta_{r}^{l}T_{t}\,+\,\delta...",Oui,"""FUND - agricultural impact of the level of cl...",FUND
3,Agriculture impact of fertilisation,A.4,FUND v.3.8,"Time,Region,Pre-industrial concentration of CO...",Damage in agricultural production due to CO2 f...,NaN,a4.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food security reduction,Food production reduction",", CO2 concentration",Production,"A_{t,r}^{f}=\nu_{r}\ln\frac{G O2_{t}}{275}\,,",Oui,"""FUND - agricultural impact of the fertilisation""",FUND
4,Agriculture production relative loss,A.5,FUND v.3.8,"Gross agricultural product,GDP in region r at ...",NaN,NaN,a5.png (https://v5.airtableusercontent.com/v3/...,[image] ([pdf](zotero://open-pdf/library/items...,"Food production reduction,Food security reduction","Production, GDP,",NaN,"\frac{G A P_{t,r}}{Y_{t,r}}=\frac{G A P_{1990,...",Oui,NaN,FUND


In [6]:
variables['Model'] = variables['Model'].str.strip()
variables['Equation'] = variables['Equation'].str.strip()
df['Model_simplified'] = df['Model_simplified'].str.strip()
df['Equation number'] = df['Equation number'].str.strip()



merged = variables.merge(df, right_on=['Model_simplified', 'Equation number'], left_on=['Model', 'Equation'])
merged.columns

Index(['Unnamed: 0', 'Real Name', 'Py Name', 'Subscripts', 'Units', 'Limits',
       'Type', 'Subtype', 'Comment', 'Model', 'Equation', 'isEquation', 'Name',
       'Equation number', 'Model - new table', 'Input variable',
       'Output variables', 'Equation shape', 'Formule', 'Zotero reference',
       'IPCC risks', 'Input categories', 'Output categories', 'Latex fragment',
       'Vensim', 'Vensim name', 'Model_simplified'],
      dtype='object')

In [7]:
merged['Input variable'] = merged['Input variable'].str.split(',')
merged['Output variables'] = merged['Output variables'].str.split(',')
merged.columns

Index(['Unnamed: 0', 'Real Name', 'Py Name', 'Subscripts', 'Units', 'Limits',
       'Type', 'Subtype', 'Comment', 'Model', 'Equation', 'isEquation', 'Name',
       'Equation number', 'Model - new table', 'Input variable',
       'Output variables', 'Equation shape', 'Formule', 'Zotero reference',
       'IPCC risks', 'Input categories', 'Output categories', 'Latex fragment',
       'Vensim', 'Vensim name', 'Model_simplified'],
      dtype='object')

In [24]:
import os

rst_file ='Damage functions\n===============\n\n'

# Filter the dataset to only equations
df = merged[merged["isEquation"]==True]


# Store the model name in a variable
current_model = None


for row in df.iterrows():

    if current_model != row[1]['Model - new table']:
        rst_file += row[1]['Model - new table'] + '\n' + '-'*len(row[1]['Model - new table']) + '\n\n'
    else:
        pass

    current_model = row[1]['Model - new table']

    rst_file += str(row[1]['Equation number']) + " : "  + row[1]['Name'] + '\n' + '~'*(3+len(row[1]['Name'])+len(str(row[1]['Equation number']))) + '\n\n'

    try: 
        rst_file += '.. math:: \n\n'
        rst_file += '   '+ row[1]['Latex fragment'] + ' \n\n'
    except:
        pass

    rst_file += 'Input variables:\n\n'

    try:
        for input in row[1]['Input variable']:
            rst_file += '- ' + input + '\n'
    except:
        pass

    rst_file += '\nOutput variables:\n\n'
    
    try: 
        for output in row[1]['Output variables']:
            rst_file += '- ' + output + '\n'
        rst_file += '\n\n'
    except:
        pass

    rst_file += '.. autofunction:: WILIAM.output.' + row[1]['Py Name'] + ' \n\n'

    #print('.. autofunction:: WILIAM.output.' + row[1]['Py Name'] + ' \n\n')


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
file_path = os.path.join(parent_dir, '4_API\\output.rst')


with open(file_path, 'w') as file:
    file.write(rst_file)
    print('File written to: ', file_path)

File written to:  c:\users\gabriel.genelot\documents\damage-functions\docs\source\4_API\output.rst


## Trying to write documentation in the mdl file to have the correct form

In [9]:
# Example documentation 

def read_vensim(mdl_file, data_files=None, initialize=True,
                missing_values="warning", split_views=False,
                encoding=None, **kwargs):
    """
    Construct a model from Vensim `.mdl` file.

    Parameters
    ----------
    mdl_file: str or pathlib.Path
        The relative path filename for a raw Vensim `.mdl` file.

    initialize: bool (optional)
        If False, the model will not be initialize when it is loaded.
        Default is True.

    data_files: list or str or None (optional)
        If given the list of files where the necessary data to run the model
        is given. Default is None.

    missing_values: str ("warning", "error", "ignore", "keep") (optional)
        What to do with missing values. If "warning" (default)
        shows a warning message and interpolates the values.
        If "raise" raises an error. If "ignore" interpolates
        the values without showing anything. If "keep" it will keep
        the missing values, this option may cause the integration to
        fail, but it may be used to check the quality of the data.

    split_views: bool (optional)
        If True, the sketch is parsed to detect model elements in each
        model view, and then translate each view in a separate Python
        file. Setting this argument to True is recommended for large
        models split in many different views. Default is False.

    encoding: str or None (optional)
        Encoding of the source model file. If None, the encoding will be
        read from the model, if the encoding is not defined in the model
        file it will be set to 'UTF-8'. Default is None.

    subview_sep: list
        Characters used to separate views and subviews (e.g. [",", "."]).
        If provided, and split_views=True, each submodule will be placed
        inside the directory of the parent view.

    **kwargs: (optional)
        Additional keyword arguments for translation.

    Returns
    -------
    model: a PySD class object
        Elements from the Python model are loaded into the PySD class
        and ready to run

    Examples
    --------
    >>> model = read_vensim('../tests/test-models/samples/teacup/teacup.mdl')

    """

    return True

In [10]:
merged.head()

,Unnamed: 0,Real Name,Py Name,Subscripts,Units,Limits,Type,Subtype,Comment,Model,...,Equation shape,Formule,Zotero reference,IPCC risks,Input categories,Output categories,Latex fragment,Vensim,Vensim name,Model_simplified
0,8,DICE: 5: EQ damage function,dice_5_eq_damage_function,NaN,NaN,"(nan, nan)",Auxiliary,Normal,NaN,DICE,...,NaN,5.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l l}{{\ }}&{{\displaystyle=\psi_...,Oui,NaN,DICE
1,9,DICE: 5: damage function phi 1,dice_5_damage_function_phi_1,NaN,NaN,"(nan, nan)",Constant,Normal,NaN,DICE,...,NaN,5.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l l}{{\ }}&{{\displaystyle=\psi_...,Oui,NaN,DICE
2,10,DICE: 5: damage function phi2,dice_5_damage_function_phi2,NaN,NaN,"(nan, nan)",Constant,Normal,NaN,DICE,...,NaN,5.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l l}{{\ }}&{{\displaystyle=\psi_...,Oui,NaN,DICE
3,11,DICE: 6: EQ abatment function,dice_6_eq_abatment_function,NaN,NaN,"(nan, nan)",Auxiliary,Normal,NaN,DICE,...,NaN,6.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l}{{\Lambda({\bf t})\;=\;\theta_...,Oui,NaN,DICE
4,12,DICE: 6: Emissions control rate,dice_6_emissions_control_rate,NaN,NaN,"(nan, nan)",Constant,Normal,TODO: checker cette valeur là,DICE,...,NaN,6.png (https://v5.airtableusercontent.com/v3/u...,[image] ([pdf](zotero://open-pdf/library/items...,NaN,NaN,NaN,\begin{array}{l}{{\Lambda({\bf t})\;=\;\theta_...,Oui,NaN,DICE


In [11]:
variable_description = pd.read_csv('../../../variables_df.csv')
variable_description['Name'] = variable_description['Name'].str.strip() 

variable_description.head(2)

,Name,Variable sign,Description,Unit,Status,Damage functions,Damage functions 2,Related equation,Zotero Source,Quantité (Damage functions),Sector,Module,Type,Scope,Formule (à partir de Damage functions),WILIAM name
0,Total agriculture impact,NaN,Impact of climate change on agriculture at tim...,NaN,NaN,NaN,Total agriculture impact,NaN,NaN,0,Production,Economy,Stock,Regional,NaN,NaN
1,Damage in agricultural production,"A_{t,r}",Damage in agricultural production as a fracti...,Fraction,NaN,Total agriculture impact,Agriculture impact of the rate of climate change,Equation A.2,NaN,1,Production,Economy,Stock,Regional,a1.png (https://v5.airtableusercontent.com/v3/...,NaN


In [12]:
def write_description(variable, variable_description=variable_description, df=merged, equation="No equation"):

    output = ''
    
    print(f'Writing description for {variable}')

    variable = variable.strip()
    eq = df[df['Real Name'] == variable]

    # Adding description
    output += '\t\tDescription' + '\n\n'

    # Adding input parameters
    output += '\t\tParameters\n\t\t----------\n'

    try: 

        for params in eq['Input variable']: 
                
            for param in params: 

                description = variable_description[variable_description['Name'] == param]

                output += '\t\t' + param.strip() + ': \n'
                if not description.empty:
                    output += '\t\t\t' +  str(description.iloc[0, description.columns.get_loc("Description")]) + '\n\n'   # param['Description'] + '\n\n'
                else:
                    output += '\t\t\t' + 'No description' + '\n\n'
        # Adding return = output variable
    except:
        pass

    output += '\t\tReturns\n\t\t-------\n'

    try: 
        for param in eq['Output variables']:

            description = variable_description[variable_description['Name'] == param]

            output_variable = eq['Output variables'].iloc[0]

            output += '\t\t' + param + ': \n'
            output += '\t\t\t' + output_variable + '\n\n'   # param['Description'] + '\n\n'  param['Description'] + '\n\n'
    except:
        output += '\t\t' + 'No variable\n'  '\t\t\t' + equation + "\n\n"

    return output

In [15]:
# Open the model file as a text file
with open("../../../WILIAM/WILIAM.mdl", 'r', encoding='latin1') as file:
    content = file.read()

    content = content.split('UTF-8}\n')[1]

# Split the content of the file at each variable
variables = content.split('|') 
modified_model ="{UTF-8}\n"

# For each variable, try to run a function that will replace the description
for variable in variables:

    variable_name = None

    try: 

        # Split the variable into its components
        caracteristiques = variable.split('~')

        # Clean variable name 
        variable_name = caracteristiques[0].split('[')[0].strip()
        #print(f'Variable name: {variable_name}')
        try:
            variable_name = variable_name.replace('"', '')
            #print("Guillements remplacés" + variable_name)
        except:
            pass

        #print(variable_name)
    except:
        pass
        #print('No variable name found')


    # Get the equation
    equation = caracteristiques[0].strip()

    #print(variable_name)

    try:
        # Check if it is a variable that has a description
        if variable_name in merged['Real Name'].values:

            # Saving the old description to avoid looping
            try: 
                old_description = caracteristiques[2].split('Description')[0]
            except:
                old_description = ''

            try:
                new_description = write_description(variable_name, variable_description, merged, equation)
                #print(new_description)
            except:
                print(f"Can't write description for {variable_name}")

            try: 
                caracteristiques[2] = old_description + '\n\n' + new_description # Watch out here because it takes into account the old description, so it could loop
            #print(f'Description added for {variable_name}')
            #print(caracteristiques[2])
            except:
                print(f"Can't add description for {variable_name}")

        # Reconstruct the variable
        modified_variable = caracteristiques[0] + '\t~\t' + caracteristiques[1] + '\t~\t' + caracteristiques[2] + '\n\t\t|\n'
        #print(modified_variable)
    except:
        pass
        #print('Keeping the old variable')
        # If not, keep the old variable
        modified_variable = variable + '|'


    modified_model += modified_variable

with open('../../../WILIAM/output.mdl', 'w', encoding='latin1') as f:
    f.write(modified_model)


Writing description for FUND: A.1: EQ Total agricultural impact
Writing description for FUND: A.2: alpha parameter
Writing description for FUND: A.2: DELAYED agricultural impact of the rate of climate change
Writing description for FUND: A.2: EQ agricultural impact of the rate of climate change
Writing description for FUND: A.2: rate of temperature change
Writing description for FUND: A.2: temperature change delayed
Writing description for FUND: A.3: delta l parameter
Writing description for FUND: A.3: EQ agricultural impact of the level of climate change
Writing description for FUND: A.4: delta q parameter
Writing description for FUND: A.4: EQ agricultural imact of the fertilisation
Writing description for FUND: A.4: gamma parameter
Writing description for FUND: E.1: alpha parameter
Writing description for FUND: E.1: EQ Space heating
Writing description for FUND: E.1: EQ value of the loss of the ecosystems
Writing description for FUND: E.2: alpha parameter
Writing description for FUND

## Trying to add many-to-many relations in the nocodb

In [46]:
import pandas as pd
import requests

# Configuration
api_url = 'https://app.nocodb.com'
project_id = 'pgxzs6196j7d22v'
table_a_id = 'mjlzlvkuysvdvuq' # functions
table_b_id = 'm0gqe4gfebcvib3' # models
api_key = 'b0GndcsjgOx20Rnva1_3Rjvp5erw9fw6u-dALzqC'

# Headers pour l'API
headers = {
    'xc-auth': api_key,
    'Content-Type': 'application/json'
}

# Charger le fichier CSV
#csv_file_path = 'path_to_your_file.csv'
data = merged.copy()

# Fonction pour ajouter une entrée à une table
def add_entry(table_id, entry):
    response = requests.post(f'{api_url}/api/v1/{project_id}/{table_id}', headers=headers, json=entry)
    return response.json()

# Fonction pour obtenir ou créer une entrée dans TableB
def get_or_create_table_b_entry(value):
    # Vérifiez si l'entrée existe déjà
    response = requests.get(f'{api_url}/api/v2/{project_id}/{table_b_id}', headers=headers, params={'where': f'Model="{value}"'})
    result = response.json()
    if result:
        return result[0]['Id']
    else:
        # Sinon, créez la nouvelle entrée
        entry = {'Model': value}
        response = add_entry(table_b_id, entry)
        return response['Id']

# Traiter les données et ajouter des entrées dans TableA avec des relations many-to-many
for index, row in data.iterrows():
    # Créez ou récupérez les relations many-to-many
    related_ids = [get_or_create_table_b_entry(value) for value in row['Model - new table'].split(',')]
    
    # Ajouter l'entrée dans TableA
    table_a_entry = {
        'Field1': row['Field1'],
        'Field2': row['Field2'],
        # Ajoutez d'autres champs ici
        'relationField': related_ids  # Remplacez relationField par le nom réel du champ de relation many-to-many
    }
    
    add_entry(table_a_id, table_a_entry)

print("Données importées avec succès dans NocoDB.")


KeyError: 0

In [2]:
import requests

access_token = "b0GndcsjgOx20Rnva1_3Rjvp5erw9fw6u-dALzqC"

# Configuration
api_url = 'https://app.nocodb.com'
project_id = 'pgxzs6196j7d22v'
table_a_id = 'mjlzlvkuysvdvuq' # functions
table_b_id = 'm0gqe4gfebcvib3' # models

s = requests.Session()

# Create row
create_row_request = requests.post(f'https://app.nocodb.com/api/v1/{project_id}/{table_a_id}',
                  headers={'Content-Type': 'application/json',
                           'xc-token': access_token,
                           },
                  json={
                      "Model": "titleB"
                  }
                  )

# Find row
find_one_request = requests.get(f'https://app.nocodb.com/api/v1/{project_id}/{table_b_id}/find-one?where=(Model,eq,titleB)',
                                headers={'Content-Type': 'application/json',
                                         'xc-token': access_token,
                                         }
                                )
if find_one_request.status_code == 200:
    # Proceed to extract 'Id'
    row_id = find_one_request.json().get("Id")
    if row_id is None:
        print("Id not found in response.")
else:
    print(f"Request failed with status code: {find_one_request.status_code}")

row_id = find_one_request.json()["Id"]

# Get row id (other table)
find_one_request2 = requests.get(f'https://app.nocodb.com/api/v1/{project_id}/{table_a_id}/find-one?where=(Model,eq,titleA)',
                                 headers={'Content-Type': 'application/json',
                                          'xc-token': access_token,
                                          }
                                 )
row_id_other_table = find_one_request2.json()["Id"]


# Link row
link_row = requests.post(f'https://app.nocodb.com/api/v1/{project_id}/{table_b_id}/{row_id}/mm/columnName/{row_id_other_table}',
                         headers={'Content-Type': 'application/json',
                                  'xc-token': access_token,
                                  }
                         )
print(f"Status Code: {link_row.status_code}, Response: {link_row.json()}")

Request failed with status code: 404


KeyError: 'Id'